In [4]:
from pathlib import Path
import re

# ====== KONFIGURASI ======
SQL_PATHS = [
    r"D:\Media\Kuliah\Skripsi\ScripTI\Versi 4\Preprocessing\deskevaluations.sql",
    "/mnt/data/deskevaluations.sql",
]

START_MARKER = """INSERT INTO `deskevaluations` (`id`, `tanggal_valid`, `catatan`, `modified`, `user_id`, `status`, `dosen1`, `dosen2`, `proposal_id`, `judul_lama`, `judul_baru`, `nim`, `upload_revisi`, `evaluator_id`, `sidang_id`, `Additional`) VALUES"""
END_MARKER = """
--
-- Indexes for dumped tables
--
""".strip("\n")  # biar lebih fleksibel di awal/akhir baris

# ====== UTIL ======
def read_all(paths):
    for p in map(Path, paths):
        if p.exists():
            for enc in ("utf-8", "utf-8-sig", "latin-1"):
                try:
                    return p.read_text(encoding=enc)
                except UnicodeDecodeError:
                    continue
            return p.read_bytes().decode("latin-1", errors="replace")
    raise FileNotFoundError("File SQL tidak ditemukan di path yang disediakan.")

def normalize_newlines(s: str) -> str:
    return s.replace("\r\n", "\n").replace("\r", "\n")

def whitespace_tolerant_regex(s: str) -> re.Pattern:
    """
    Buat regex yang menganggap semua whitespace di 's' bisa jadi
    spasi/enter/tab berapa pun. Karakter lain di-escape supaya literal.
    """
    # Normalize whitespace di marker dulu
    s = normalize_newlines(s)
    # Pisah berdasar whitespace lalu sambung dengan \s+
    parts = re.split(r"\s+", s.strip())
    pattern = r"\s*".join(map(re.escape, parts))
    return re.compile(pattern, flags=re.IGNORECASE | re.DOTALL)

def find_first(haystack: str, needle: str) -> int:
    """Cari needle persis; kalau gagal, pakai regex toleran whitespace."""
    idx = haystack.find(needle)
    if idx >= 0:
        return idx
    # fallback regex
    m = whitespace_tolerant_regex(needle).search(haystack)
    return m.start() if m else -1

def find_between_markers(text: str, start_marker: str, end_marker: str) -> str:
    t = normalize_newlines(text)
    sm = normalize_newlines(start_marker).strip()
    em = normalize_newlines(end_marker).strip()

    i = find_first(t, sm)
    if i < 0:
        # Tidak ada pembuka -> kembalikan apa adanya
        return t
    # Mulai setelah pembuka (pakai panjang pembuka versi persis kalau ada,
    # kalau tidak, gunakan panjang kecocokan regex)
    # Untuk akurat, cari lagi rentang akhir pembuka dengan regex agar tahan whitespace
    sm_re = whitespace_tolerant_regex(sm)
    m = sm_re.search(t, i)
    start_after = m.end() if m else i + len(sm)

    j = find_first(t[start_after:], em)
    if j < 0:
        # Tidak ada penutup -> ambil dari setelah pembuka sampai akhir
        return t[start_after:]
    end_index = start_after + j
    return t[start_after:end_index]

# ====== MAIN ======
if __name__ == "__main__":
    whole = read_all(SQL_PATHS)
    data = find_between_markers(whole, START_MARKER, END_MARKER)

    # (Opsional) simpan hasil slice ke file .sql untuk verifikasi
    out = Path("between_insert_and_indexes.sql")
    out.write_text(data, encoding="utf-8")

    # Bungkus ke variabel Python 'data' kalau mau dipakai di script lain
    py_out = Path("./DataEvaluations.py")
    py_out.write_text('data = """\n' + data.replace('"""', r'\"\"\"') + '\n"""', encoding="utf-8")

    print(f"OK. Panjang hasil: {len(data):,} chars")
    print(f"Simpan: {out.resolve()}")
    print(f"Juga file Python: {py_out.resolve()}")


OK. Panjang hasil: 2,304,596 chars
Simpan: D:\Media\Kuliah\Skripsi\ScripTI\Versi 4\Preprocessing\between_insert_and_indexes.sql
Juga file Python: D:\Media\Kuliah\Skripsi\ScripTI\Versi 4\Preprocessing\DataEvaluations.py


In [ ]:
import re
import html
import unicodedata as ud
from DataEvaluations import data

def is_alpha(ch):
    return bool(ch) and (ch.isalpha() or ud.category(ch).startswith('L'))

def is_numeric(ch): 
    return bool(ch) and (ch.isdigit() or ud.category(ch).startswith('N'))

def is_space_or_punct_or_symbol(ch: str) -> bool:
    cat = ud.category(ch)     # contoh: 'Ll','Lu','Nd','Ps','Pe','Zs','Po','Sm','So',...
    return cat[0] in ('Z', 'P', 'S')

def removeHTMLString(data):
    
    #Komleks Taq HTML
    data = data.replace('<p class=\"MsoNormal\" style=\"margin-left:.25in;text-align:justify;text-indent:\r\n.25in;line-height:150%\"><span style=\"font-family:times new roman,serif; font-size:12.0pt; line-height:150%\">',"")
    data = data.replace("<o:p></o:p>", "")
    data = data.replace("<p class=\"MsoNormal\" style=\"margin-left:.25in;text-align:justify;text-indent:\r\n.25in;line-height:150%\">", "")
    data = data.replace("</p>\r\n\r\n<p><!--[if supportFields]><b style=\'mso-bidi-font-weight:normal\'><span lang=ZH-CN\r\nstyle=\'font-size:12.0pt;line-height:107%;font-family:\"Times New Roman\",serif;\r\nmso-fareast-font-family:Cambria;mso-ansi-language:ZH-CN;mso-fareast-language:\r\nZH-CN;mso-bidi-language:AR-SA\'><span style=\'mso-element:field-end\'></span></span></b><![endif]--></p>\r\n","")
    data = data.replace("<p><!--[if supportFields]><b style=\'mso-bidi-font-weight:\r\nnormal\'><span lang=ZH-CN style=\'font-size:12.0pt;line-height:107%;font-family:\r\n\"Times New Roman\",serif;mso-fareast-font-family:Cambria\'><span\r\nstyle=\'mso-element:field-begin\'></span></span></b><b style=\'mso-bidi-font-weight:\r\nnormal\'><span style=\'font-size:12.0pt;line-height:107%;font-family:\"Times New Roman\",serif;\r\nmso-fareast-font-family:Cambria;mso-ansi-language:EN-US\'><span\r\nstyle=\'mso-spacerun:yes\'> </span>BIBLIOGRAPHY<span style=\'mso-spacerun:yes\'> \r\n</span>\\l 1033 </span></b><b style=\'mso-bidi-font-weight:normal\'><span\r\nlang=ZH-CN style=\'font-size:12.0pt;line-height:107%;font-family:\"Times New Roman\",serif;\r\nmso-fareast-font-family:Cambria\'><span style=\'mso-element:field-separator\'></span></span></b><![endif]-->","")
    data = data.replace("<!--[if supportFields]><b style=\'mso-bidi-font-weight:\r\nnormal\'><span style=\'font-size:12.0pt;line-height:150%;font-family:\"Times New Roman\",serif;\r\nmso-fareast-font-family:\"Times New Roman\";color:black\'><span style=\'mso-element:\r\nfield-begin;mso-field-lock:yes\'></span>","")
    data = data.replace("<span style=\'mso-element:field-separator\'></span></span></b><![endif]-->","")
    data = data.replace("</p>\r\n\r\n<p><!--[if supportFields]><b\r\nstyle=\'mso-bidi-font-weight:normal\'><span style=\'font-size:12.0pt;line-height:\r\n150%;font-family:\"Times New Roman\",serif;mso-fareast-font-family:\"Times New Roman\";\r\ncolor:black\'><span style=\'mso-element:field-end\'></span></span></b><![endif]--><strong>&nbsp;</strong></p>\r\n", "")
    data = data.replace("INSERT INTO `deskevaluations` (`id`, `tanggal_valid`, `catatan`, `modified`, `user_id`, `status`, `dosen1`, `dosen2`, `proposal_id`, `judul_lama`, `judul_baru`, `nim`, `upload_revisi`, `evaluator_id`, `sidang_id`, `Additional`) VALUES", "")
    data = data.replace(":</p>\r\n\r\n<p>", ": ")
    data = data.replace(")</p>\r\n\r\n<p>", "). ")
    data = data.replace(".</p>\r\n\r\n<p>", ". ")
    data = data.replace("</p>\r\n\r\n<p>", " ")
    data = data.replace(":\r\na", ": ")

    #Vairasi Taq P
    data = data.replace("<p>?</p>","")
    data = data.replace("1.</p>","")
    data = data.replace(".?</p>",".")
    data = data.replace(".</p>",".")
    data = data.replace(")</p>",").")
    data = data.replace('</p>',"")
    data = data.replace("</p>","")
    data = data.replace("<p>","")
    data = data.replace("./p>",".")
    
    #Variasi Entitas
    data = data.replace("&#39;","`")
    data = data.replace("&hellip;..","")
    data = data.replace("&nbsp;"," ")
    data = data.replace("&iuml;", "ï")
    data = data.replace("&igrave;;","ì")
    data = data.replace("&uuml;","ü")
    data = data.replace("&ouml;","ö")
    data = data.replace("&iacute;","í")
    data = data.replace("&egrave;","è")
    data = data.replace("&eacute;","é")
    data = data.replace("&agrave;","à")
    data = data.replace("v&gt;","")
    
    
    #Variasi Gagal Encoding
    data = data.replace("p?ny?n", "pinyin")
    data = data.replace("h?nz?", "hanzi")
    
    #Variasi Strong
    data = data.replace("<strong>", "")
    data = data.replace("/strong","")
    
    #Variasi ol li
    data = data.replace("<ol><li><em>","")
    data = data.replace("<li><em>",", ")
    data = data.replace(".</li>\r\n",", ")
    data = data.replace("</li>","")
    data = data.replace("<li>"," ")
    data = data.replace("<ul>", "")
    data = data.replace("</ul>", "")
    data = data.replace("<ul", "")
    data = data.replace("</ul", "")
    data = data.replace("<ol>","")
    data = data.replace("</ol>","")
    data = data.replace("<>","")

    #Variasi T
    data = data.replace("\t1", "")
    data = data.replace("\t2", "")
    data = data.replace("\t", "")

    #Variasi Lainnya    
    data = data.replace("\r","")
    data = data.replace("</span>","")
    data = data.replace("<span>","")
    data = data.replace("<br />","")
    data = data.replace("<br/>","")
    data = data.replace("</em>", "")
    data = data.replace("<em>", "")
    data = data.replace("\n","")
    data = data.replace("/b","")
    
    data = _fix_em_dot_rule(data)

    # ---- Hilangkan penanda [number] (mis. [1], [23]) ----
    data = re.sub(r'\[\s*\d+\s*\]\s*', '', data)
    data = re.sub(r'\.\s*([A-Za-z])\s*\.', r'. \1.', data)
    # data = data.replace("</em>.", ",")

    data = re.sub(r'([.!?])(?:\s*[.!?])+(?=\s|$)', r'\1', data)
    data = re.sub(r'<[^>]+>', '', data)
    data = html.unescape(data)
    
    return data

import re

def _fix_em_dot_rule(text: str) -> str:
    """
    Ubah pola:  <word></em>. <word>
    sesuai aturan kapitalisasi yang diminta.
    """
    # word = huruf awal (Unicode letter) lalu susulan \w/- (biar aman utk bahasa Indo/Inggris)
    # gunakan UNICODE agar .isupper()/islower() relevan
    pat = re.compile(r'(?P<left>\b[^\W\d_][\w\-]*)\s*</em>\.\s+(?P<right>[^\W\d_][\w\-]*)', re.UNICODE)

    out = []
    last = 0
    for m in pat.finditer(text):
        lw = m.group('left')
        rw = m.group('right')

        l_upper = lw[0].isupper()
        l_lower = lw[0].islower()
        r_upper = rw[0].isupper()
        r_lower = rw[0].islower()

        # default: boundary kalimat
        sep = ". "
        new_lw = lw

        if l_lower and r_lower:
            sep = ", "
            new_lw = lw
        elif l_upper and r_upper:
            sep = ". "
            new_lw = lw
        elif l_upper and r_lower:
            sep = ", "
            # decapitalize kata sebelum
            new_lw = lw[0].lower() + lw[1:]
        elif l_lower and r_upper:
            sep = ". "
            new_lw = lw

        # tambahkan bagian sebelum match
        out.append(text[last:m.start('left')])
        # susun pengganti
        out.append(new_lw + sep + rw)
        last = m.end('right')

    out.append(text[last:])
    return "".join(out)




def splitField(rawData): #Data String Besar
    # Memecah string besar menjadi potongan-potongan data per-insert.
    # Setiap elemen dalam list ini berisi 9 data DeskEvaluation.
    dataPerInsert = []

    # Memecah 9 data DeskEvaluation dari setiap elemen dataPerInsert
    # menjadi list yang setiap elemennya adalah 1 data DeskEvaluation.
    DeskEvaluationList = []

    # Mengelompokkan data DeskEvaluation yang telah dipisahkan
    # berdasarkan ID-nya.
    groupedDeskEvaluationsById = []

    # Memisahkan setiap data DeskEvaluation menjadi bagian-bagian yang lebih kecil
    # sesuai struktur field. Namun, beberapa field yang seharusnya digabung
    # ikut terpisah.
    rawSplitFields = []

    # Menggabungkan kembali bagian-bagian field yang terpisah dari
    # rawSplitFields.
    recombinedFields = []

    dataFinal = [] #Data DeskEvaluation yang sudah sesuai struktur DeskEvaluation
    kalimat = "" #Variabel untuk menampung kalimat yang terpisah untuk digabung kembali
    kalimatSebelumnya = "" #Variabel untuk menampung kalimat sebelumnya yang ketika sudah lengkap akan dimasukkan ke dalam variable kalimat sebelum di insert ke data DeskEvaluation
    StringBuka = False #Variable untuk mendeteksi apakah ada tanda ' yang belum tertutup dalam sebuah string

    totalRemoveDeskEvaluation = 0 #Variabel untuk menampung jumlah DeskEvaluation yang dihapus karena judul kosong
    dataPerInsert = rawData.split("');")
    for i in dataPerInsert :
        DeskEvaluationList = i.split("'),")
        for j in DeskEvaluationList :
            groupedDeskEvaluationsById.append(j)

    for i in groupedDeskEvaluationsById :
        if len(recombinedFields) == 16 :
            # if recombinedFields[0] == '1169':
            #     print("debug")
            # if recombinedFields[0] == '(380' :
            #     print("debug")
            if recombinedFields[5] != " 'T'" :
                totalRemoveDeskEvaluation = totalRemoveDeskEvaluation + 1
                # print(recombinedFields[0])
                recombinedFields = []
            else :
                index = 0
                for field in recombinedFields :
                    listChar = list(field)
                    for indexChar in range(len(listChar)) :
                        if is_alpha(listChar[indexChar]) or is_numeric(listChar[indexChar]):
                            recombinedFields[index] = field
                            index = index + 1
                            break
                        if  (is_space_or_punct_or_symbol(listChar[indexChar]) or listChar[indexChar] in ("'", '"', ".", ",", ":", ";", "!", "?", "(", ")", "[", "]", "{", "}", "<", ">", "-", "_", "/", "\\", "&", "%", "$", "#", "@", "*", "+", "=", "~", "`", "^")):
                            EXCLUDE = ("'", '"')  # yang TIDAK dihapus
                            if listChar[indexChar] not in EXCLUDE:
                                listChar[indexChar] = ""  # cek bukan kutip
                                field = "".join(listChar)
                            else :
                                if is_alpha(listChar[indexChar+1]) or is_numeric(listChar[indexChar+1]):
                                    recombinedFields[index] = field
                                    index = index + 1
                                    break
                                    # print(listChar[indexChar+1])
                                else :
                                    listChar[indexChar+1] = ""  # cek bukan kutip
                                    field = "".join(listChar)
                                    recombinedFields[index] = field
                                    index = index + 1
                                    break
                dataFinal.append(recombinedFields)

                recombinedFields = []
        debug  = "sabar"
        debugIndex = 0
        rawSplitFields = i.split(",")
        for j in rawSplitFields  :
            if (rawSplitFields[len(rawSplitFields)- 1] == j):
                recombinedFields.append(j)
            else :

                if j == '(3328' :
                    debug = "gas"
                    print("debug")
                if debug == "gas" :
                    debugIndex = debugIndex + 1
                    if debugIndex == 13 :
                        print("debug")
                if len(j) != 0:
                    if j.count("'") == 1 and StringBuka == False: #Mendeteksi apakah Kutip String belun dibuka dan hanya terdapat 1 kutip untuk menandakan bahwa kalimat itu memiliki kelanjutan sehingga hasil akhir kalimat tidak akan langsung diinputkan ke dalam data DeskEvaluation
                        kalimat = kalimat + ", " + j #kalimat pertama masuk ke sini
                        StringBuka = True
                    elif j.count("'") == 0 and StringBuka == True: #Mendeteksi apakah kutip string sudah dibuka dan tidak ada kutip pada kata tersebut untuk menandakan kalimat akan terus berlanjut atau memilikii kelanjutan
                        listCharKalimat = list(j) #Pengecekan apakah ini merupakan kalimat yang akhir atau hanya kalimat yang terpisah hasil split tanda (,)
                        for indexChara in range(len(listCharKalimat)) :
                            if is_alpha(listCharKalimat[indexChara]) or is_numeric(listCharKalimat[indexChara]):
                                kalimat = kalimat + ", " + j
                                break
                            if  (is_space_or_punct_or_symbol(listCharKalimat[indexChara]) or listCharKalimat[indexChara] in ("'", '"', ".", ",", ":", ";", "!", "?", "(", ")", "[", "]", "{", "}", "<", ">", "-", "_", "/", "\\", "&", "%", "$", "#", "@", "*", "+", "=", "~", "`", "^")):
                                EXCLUDE = ("'", '"','.')  # yang TIDAK dihapus
                                if listCharKalimat[indexChara] not in EXCLUDE:
                                    listCharKalimat[indexChara] = ""
                                else:
                                    aha = "".join(listCharKalimat)
                                    kalimat = kalimat + aha
                                    break
                    elif j.count("'") >= 1 and StringBuka == True and (re.findall(r"\w+'\w+|\w+'\s\w+|'\w+|\b\w+'\w*\b|'\w[\w\s]*?'|'\w+",j)) : #Mendeteksi apakah kutip string sudah dibuka dan dalam kalimat terdeteksi lebih atau setidaknya terdapat 1 kutip dalam kalimat, kemudian mendeteksi apakah kutip itu merepakan kutip penutup atau kutip dari nama orang, jalan, tempat atau suatu kata sehingga menyatakan kalimat belum berakhir)
                        if  (j[-1]=="'" and j[-2]==".") : #Mendeteksi apakah kutip itu merupakan kutip penutup kalimat dengan melihat adanya titik sebelum kutip penutup
                            kalimatSebelumnya = "" + kalimat
                            kalimat = kalimat + ", " + j
                            recombinedFields.append(kalimat)
                            # writer.writerow([recombinedFields[0], kalimat])
                            kalimat = ""
                            StringBuka = False

                        else  :
                            kalimat = kalimat + ", " + j
                    elif j.count("'") == 1 and StringBuka == True and ( not re.findall(r"\w+'\w+|\w+'\s\w+",j) or not re.findall(r"\b\w+'\w*\b|'\w[\w\s]*?'",j)) : #Mendeteksi apakah hanya terdapat 1 kutip dan kutip itu berada dipaling belakang sebagai penutup kalimat
                        kalimatLast ="" #Untuk mengatasi kelebihan tanda baca , pada akhir kalimat
                        if len(j) < 4:
                            for k in j :
                                if k != " ":
                                    kalimatLast = kalimatLast + k
                            recombinedFields.append(kalimat + kalimatLast)
                            kalimat = ""
                            StringBuka = False
                            continue
                        recombinedFields.append(kalimat + ", " + j)
                        # writer.writerow([recombinedFields[0], kalimat])
                        kalimat = ""
                        StringBuka = False
                    elif j.count("'") == 2 : #Medeteksi apa bila terdeteksi 2 kutip dalam 1 kalimat sebagai pembuka dan penutup, bila terdapat 2  dan bukan penutup atau pembuka dia akan masuk pada elif kedua terlebih dahulu, biasa kalimat pendek masuk ke dalam sini
                        recombinedFields.append(j) #judul masuk disini
                    elif j.count("'") == 0 and j[len(j)-1].isdigit() and j[0].isdigit() : #Mendeteksi apabila value dalam string hanya berupa angka dan bukan string alfabet
                        recombinedFields.append(j)
                    else : #Diluar dari semua itu akan dianggap sebagai sebuah field yang tidak terdeteksi sama sekali karena alasan tertentu
                        recombinedFields.append(j)
    print("Total Remove DeskEvaluation: ", totalRemoveDeskEvaluation)
    return dataFinal
daftarDeskEvaluation = []
data = "INSERT INTO `deskevaluations` (`id`, `tanggal_valid`, `catatan`, `modified`, `user_id`, `status`, `dosen1`, `dosen2`, `proposal_id`, `judul_lama`, `judul_baru`, `nim`, `upload_revisi`, `evaluator_id`, `sidang_id`, `Additional`) VALUES" + data
data = removeHTMLString(data) #Penghapusan HTML tag pada data
daftarDeskEvaluation = splitField(data)
print("Jumlah data setelah diproses: ", len(daftarDeskEvaluation))


debug
debug
Total Remove DeskEvaluation:  2147
Jumlah data setelah diproses:  1289


In [15]:
daftarDosenPembimbing = []
for DeskEvaluation in daftarDeskEvaluation :
    daftarDosenPembimbing.append([DeskEvaluation[8],DeskEvaluation[6],DeskEvaluation[7]])


In [16]:
print("Debug")

Debug


In [17]:
import pandas as pd
import os

# --- KONFIGURASI PATH ---
# Kita simpan di folder yang sama dengan file sistem lainnya agar terpusat
OUTPUT_DIR = r"D:\Media\Kuliah\Skripsi\ScripTI\Versi 4\Preprocessing"
os.makedirs(OUTPUT_DIR, exist_ok=True) # Buat folder jika belum ada

OUTPUT_FILE = os.path.join(OUTPUT_DIR, "proposal_dosen_ids.csv")

# --- PROSES PENYIMPANAN ---
# 1. Konversi List ke DataFrame
# Pastikan urutan kolom sesuai dengan isi list kamu: [idProposal, idDosen1, idDosen2]
df_dosen = pd.DataFrame(
    daftarDosenPembimbing, 
    columns=["proposal_id", "id_dosen_1", "id_dosen_2"]
)

# 2. Bersihkan Data (Opsional tapi disarankan)
# Kadang idDosen2 kosong/None, kita pastikan formatnya konsisten
# Mengubah string 'None', 'null', atau string kosong menjadi NaN (Not a Number) agar dikenali Pandas
df_dosen.replace(['', 'None', 'null', 'NULL'], pd.NA, inplace=True)

# 3. Simpan ke CSV
# index=False agar tidak ada kolom index tambahan (0, 1, 2...)
df_dosen.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

print(f"✅ Berhasil menyimpan {len(df_dosen)} data pemetaan dosen.")
print(f"📂 Lokasi file: {OUTPUT_FILE}")
print("\nContoh 5 data teratas:")
print(df_dosen.head())

✅ Berhasil menyimpan 1289 data pemetaan dosen.
📂 Lokasi file: D:\Media\Kuliah\Skripsi\ScripTI\Versi 4\Preprocessing\proposal_dosen_ids.csv

Contoh 5 data teratas:
  proposal_id id_dosen_1 id_dosen_2
0          26         50         10
1          52         13          8
2          51         15          5
3          63         13          8
4          98         50         41
